A simplified version of https://github.com/pytorch/examples/tree/master/word_language_model

In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.cuda as cuda

[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~']

In [19]:
import os
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        
        # This is very english language specific
        # We will ingest only these characters:
        self.whitelist = [chr(i) for i in range(32, 127)]
        
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r',  encoding="utf8") as f:
            tokens = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r',  encoding="utf8") as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids

In [3]:
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):
        
        super(RNNModel, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.drop = nn.Dropout(dropout)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, vocab_size)

        self.init_weights()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.num_layers, bsz, self.hidden_size).zero_())
    

In [4]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [20]:
corpus = Corpus('./data/shakespear')

In [21]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    if cuda.is_available():
        data = data.cuda()
    return data

In [22]:
bs_train = 20       # batch size for training set
bs_valid = 10       # batch size for validation set
bptt_size = 35      # number of times to unroll the graph for back propagation through time
clip = 0.25         # gradient clipping to check exploding gradient

embed_size = 200    # size of the embedding vector
hidden_size = 200   # size of the hidden state in the RNN 
num_layers = 2      # number of RNN layres to use
dropout_pct = 0.3   # %age of neurons to drop out for regularization

In [23]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

74010


In [24]:
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)

if cuda.is_available():
    model.cuda()

In [25]:
criterion = nn.CrossEntropyLoss()

In [26]:
train_data = batchify(corpus.train, bs_train)
val_data = batchify(corpus.valid, bs_valid)

In [27]:
train_data.shape

torch.Size([51995, 20])

In [28]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [29]:
def get_batch(source, i, evaluation=False):
    seq_len = min(bptt_size, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    return data, target

In [30]:
def train():
    # Turn on training mode which enables dropout.
    
    model.train()
    total_loss = 0
    hidden = model.init_hidden(bs_train)
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt_size)):
        
        data, targets = get_batch(train_data, i)

        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.data
        
        """
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss[0] / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        """

In [31]:

def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0
    hidden = model.init_hidden(bs_valid)
    
    for i in range(0, data_source.size(0) - 1, bptt_size):
        data, targets = get_batch(data_source, i, evaluation=True)
        output, hidden = model(data, hidden)
        output_flat = output.view(-1, vocab_size)
        total_loss += len(data) * criterion(output_flat, targets).data
        hidden = repackage_hidden(hidden)
    return total_loss[0] / len(data_source)


In [32]:
# Loop over epochs.
lr = 20
best_val_loss = None

In [33]:
for epoch in range(0, 10):
    # epoch_start_time = time.time()
    train()
    val_loss = evaluate(val_data)
    print(val_loss)
    
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "./4.model.pth")
            
    else:
        print('Annealing')
        lr /= 2.0
        
    """
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                       val_loss, math.exp(val_loss)))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open(args.save, 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    """

6.8259957180305895
6.566772114474929
6.47431495880637
6.4398405471460105
6.420965882213813
6.411151598470514
6.390628079667297
6.394153066855881
Annealing
6.347571867116052
6.35412724692526
Annealing


In [34]:
num_words = 200
temperature = 1

In [35]:
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)
model.load_state_dict(torch.load("./4.model.pth"))

if cuda.is_available():
    model.cuda()
    
model.eval()

RNNModel(
  (encoder): Embedding(74010, 200)
  (drop): Dropout(p=0.3)
  (rnn): GRU(200, 200, num_layers=2, dropout=0.3)
  (decoder): Linear(in_features=200, out_features=74010, bias=True)
)

In [37]:
hidden = model.init_hidden(1)
idx = corpus.dictionary.word2idx['Thou']
input = Variable(torch.LongTensor([[idx]]).long(), volatile=True)
input.data = input.data.cuda()

print(corpus.dictionary.idx2word[idx], '', end='')

for i in range(num_words):
    output, hidden = model(input, hidden)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.data.fill_(word_idx)
    word = corpus.dictionary.idx2word[word_idx]

    #print(word + ('\n' if i % 20 == 19 else ' '), end='')
    if word == '<eos>':
        print('')
    else:
        print(word + ' ', end='')

Thou art a work; thou but so as the 
account of 'Zounds, though I, speaking in be in thy crying physic, 
71 do's I had known my sin to hear a perfect disposition for my sake. 
MOTH. Ay, sir, if she were in a little house, thyself? You'll here take 
a token too. 
FLORIZEL. I should be a base hour in Orlando. Now will be 
said, obedience, or it shall to abominable with her. Give you 
any coffin, or you be extinct with Tyre. O thou art mad 

Kings, Nuncle servants, my voices and sense. 
FALSTAFF. Zounds, By thee, sir. 

WATCH. 
I am, but so, yet scorn, did make Caesar's shirt; he won 
himself of some anon. He knows not there. You, Pistol, you 
make me. I tell me, God, she's in a judgment of the witch! 
[Exeunt.] 

SCENE V. Another hall of the forest 

Alarum. Enter CAESAR, VAUX les LEPIDUS, LORDS, and others, with a ass's Fortinbras 
on a a digestions, into a feavour, and stone, enter a foul fiend! 
SIR TOBY. O Naples follows thy a lady 